In [ ]:
import os
print(os.getcwd())

In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from gensim.models import Word2Vec
import numpy as np

!ls
# Load data
data = pd.read_json('../../src/bazinga/database/Clean_Headlines.json', lines=True)
headlines = data['headline']
labels = data['is_sarcastic']
data.info()

c:\Users\User\anaconda3\envs\GPU_Py_12_3_ML\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
'ls' is not recognized as an internal or external command,
operable program or batch file.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26709 entries, 0 to 26708
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   article_link  26709 non-null  object
 1   headline      26709 non-null  object
 2   is_sarcastic  26709 non-null  int64 
 3   clean_text    26709 non-null  object
dtypes: int64(1), object(3)
memory usage: 834.8+ KB


In [2]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [3]:
# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

def get_bert_embeddings(texts, batch_size=32):
    
   

    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Use [CLS] token and move back to CPU
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

bert_embeddings = get_bert_embeddings(headlines.tolist())

c:\Users\User\anaconda3\envs\GPU_Py_12_3_ML\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [ ]:
# Save BERT embeddings to file
np.save('bert_embeddings.npy', bert_embeddings)
bert_df = pd.DataFrame(bert_embeddings)
bert_df.to_csv('bert_embeddings.csv', index=False)

# Load BERT embeddings from file
loaded_bert_embeddings = np.load('bert_embeddings.npy')
loaded_bert_df = pd.read_csv('bert_embeddings.csv')
loaded_bert_embeddings_from_csv = loaded_bert_df.to_numpy()

In [ ]:
print(bert_embeddings)

In [ ]:
# Tokenize the headlines
tokenized_headlines = [headline.split() for headline in headlines]

# Train Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_headlines, vector_size=300, window=5, min_count=1, workers=4)

# Generate Word2Vec embeddings
def get_w2v_embeddings(texts, model):
    embeddings = []
    for text in texts:
        words = text.split()
        word_vectors = [model.wv[word] for word in words if word in model.wv]
        if word_vectors:
            embeddings.append(np.mean(word_vectors, axis=0))
        else:
            embeddings.append(np.zeros(300))  # Handle empty cases
    return np.array(embeddings)

w2v_embeddings = get_w2v_embeddings(headlines, w2v_model)

In [ ]:
# Save Word2Vec embeddings
np.save('w2v_embeddings.npy', w2v_embeddings)
w2v_df = pd.DataFrame(w2v_embeddings)

# Save to CSV
w2v_df.to_csv('w2v_embeddings.csv', index=False)
loaded_w2v_embeddings = np.load('w2v_embeddings.npy')
loaded_w2v_df = pd.read_csv('w2v_embeddings.csv')
loaded_w2v_embeddings = loaded_w2v_df.to_numpy()

In [ ]:
print(w2v_embeddings)